# -1) imports

In [1]:
from pyspark.sql import SparkSession

# 1) configurations

In [2]:
master  = "spark://bigdataspark000.xynzil3hn5duhdxf2phc4yyyne.ax.internal.cloudapp.net:7077"
appName = "Esilv"

# 2) création d'une sparkSession

In [3]:
session_builder = SparkSession.builder
session_builder.master(master)
session_builder.appName(appName)
spark_session = session_builder.getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/14 12:40:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 3) récupération du sparkContext

In [4]:
spark_context = spark_session.sparkContext
sc = spark_context

In [5]:
sc

<SparkContext master=spark://bigdataspark000.xynzil3hn5duhdxf2phc4yyyne.ax.internal.cloudapp.net:7077 appName=Esilv>

# 4) Lecture d'un fichier sur le filesystem des ordinateurs du cluster
=> Attention : le fichier doit exister au même endroit sur chaque noeud du réseau  
=> c'est pour ça qu'on utilise hdfs => on peut avoir un ajustement automatique des noeuds et des blocs lus

In [6]:
path_to_readme = "/home/student/spark/spark-3.2.0-bin-hadoop3.2/README.md"

In [7]:
rdd = spark_context.textFile(path_to_readme)

In [8]:
rdd.count()

109

# 5) lecture d'un fichier sur hdfs

In [12]:
hdfs_path = "hdfs://bigdataspark000.westeurope.cloudapp.azure.com:8020/README.txt"
rdd_hdfs_2 = spark_context.textFile(hdfs_path)

In [13]:
rdd_hdfs_2.count()

7

# 6) Lecture de la base x, et requête SQL (p 189 du du support de cours)

In [14]:
hdfs_path = "hdfs://10.0.0.4:8020/base_x.csv"

tentatitive = spark_session.read.options(
                                            header="true", 
                                            delimiter=",", 
                                            quote="\"", 
                                            mode="FAILFAST", 
                                            inferSchema="true").csv(hdfs_path)

In [15]:
print((tentatitive.count(), len(tentatitive.columns)))

(39045, 3418)


## 6.1) enregistrement en tant que table temporaire

In [16]:
tentatitive.registerTempTable("base_x")

/home/student/anaconda3/lib/python3.9/site-packages/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(
22/01/14 12:41:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


## 6.2) exécution d'une requête sql 

In [17]:
requete_sql = "Select count(*) from base_x"

In [18]:
spark_session.sql(requete_sql).collect()

[Row(count(1)=39045)]

# 7) Analyse des Cyclistes (p224 du cours)


## 7.1) lecture des cyclistes

In [19]:
hdfs_path = "hdfs://10.0.0.4:8020/cycliste_cyclistes.csv"

cyclistes = spark_session.read.options(
                                            header="true", 
                                            delimiter=",", 
                                            quote="\"", 
                                            mode="FAILFAST", 
                                            inferSchema="true").csv(hdfs_path)

In [20]:
cyclistes.count()

4829

## 7.2) one-hot encoding

In [21]:
from pyspark.ml.feature import StringIndexer

In [57]:
string_indexer = StringIndexer()

In [58]:
string_indexer = string_indexer.setInputCol("sexe").setOutputCol("sex-num")

In [59]:
string_indexer_modele = string_indexer.fit(cyclistes)

In [60]:
cyclistes_double = string_indexer_modele.transform(cyclistes)

In [61]:
cyclistes_double.show(2) # on observe la création de la colonne supplémentaire 

+--------------+-------+---+-----+-------------+-------------+-------+-------+
|      cycliste|sportif|age| sexe|        nb_km|      vitesse|attente|sex-num|
+--------------+-------+---+-----+-------------+-------------+-------+-------+
|cycliste_azrc5|   -0.5| 24|femme|14.9038044082|12.0815937384|    0.5|    0.0|
|cycliste_azrs0|   -0.5| 67|femme|14.5112928217|14.0640955863|    0.5|    0.0|
+--------------+-------+---+-----+-------------+-------------+-------+-------+
only showing top 2 rows



# 7.3) création d'une colonne de type 'vector'

In [27]:
from pyspark.ml.feature import VectorAssembler

In [62]:
vector_assembler = VectorAssembler()

In [63]:
colonnes_a_assembler = ["sportif", "age", "nb_km", "vitesse"]

In [64]:
vector_assembler = vector_assembler.setInputCols(colonnes_a_assembler)
vector_assembler = vector_assembler.setOutputCol("features")

In [65]:
points = vector_assembler.transform(cyclistes_double)

In [66]:
points.show(2)

+--------------+-------+---+-----+-------------+-------------+-------+-------+--------------------+
|      cycliste|sportif|age| sexe|        nb_km|      vitesse|attente|sex-num|            features|
+--------------+-------+---+-----+-------------+-------------+-------+-------+--------------------+
|cycliste_azrc5|   -0.5| 24|femme|14.9038044082|12.0815937384|    0.5|    0.0|[-0.5,24.0,14.903...|
|cycliste_azrs0|   -0.5| 67|femme|14.5112928217|14.0640955863|    0.5|    0.0|[-0.5,67.0,14.511...|
+--------------+-------+---+-----+-------------+-------------+-------+-------+--------------------+
only showing top 2 rows



## 7.4) renommange de la colonne à prédire en nom "labels"

In [33]:
points = points.withColumnRenamed("sex-num", "label")

In [34]:
points.show(2)

+--------------+-------+---+-----+-------------+-------------+-------+-----+--------------------+
|      cycliste|sportif|age| sexe|        nb_km|      vitesse|attente|label|            features|
+--------------+-------+---+-----+-------------+-------------+-------+-----+--------------------+
|cycliste_azrc5|   -0.5| 24|femme|14.9038044082|12.0815937384|    0.5|  0.0|[-0.5,24.0,14.903...|
|cycliste_azrs0|   -0.5| 67|femme|14.5112928217|14.0640955863|    0.5|  0.0|[-0.5,67.0,14.511...|
+--------------+-------+---+-----+-------------+-------------+-------+-----+--------------------+
only showing top 2 rows



## 7.5) création d'un jeu d'apprentissage et d'un jeu de tests

In [35]:
jeu_d_apprentissage, jeu_de_test = points.randomSplit([0.8, 0.2])

# 7.6) modélisation

In [36]:
from pyspark.ml.classification import DecisionTreeClassifier

In [37]:
algo = DecisionTreeClassifier()

In [67]:
modele_decision_tree = algo.fit(jeu_d_apprentissage)

## 7.7) usage du modèle

In [68]:
predictions = modele_decision_tree.transform(jeu_de_test)

In [71]:
predictions.select("prediction").show(20)

+----------+
|prediction|
+----------+
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       1.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
|       0.0|
+----------+
only showing top 20 rows



## 7.8) évaluation de la prédiction 

In [42]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [43]:
evaluator = BinaryClassificationEvaluator()

In [44]:
evaluator.evaluate(predictions)

0.688919698314108

In [16]:
rdd_hdfs.take(5)

[',aag.schlau,aag.schlaudanke,adac.aktuelleverkehrslage,adac.alpenstrassen,adac.alternativekraftstoffe,adac.andertankstelle,adac.auslandlzhome,adac.auslandlzkont,adac.auslandlzrechner,adac.auslkrankschkontaktfaq,adac.auslkrankschrechnerantrag,adac.auslkrankschueberblick,adac.auslkrankschuleistungen,adac.auslkrankschverteiler,adac.autobahneroeffnungen,adac.autodaten,adac.autodatenautokosten,adac.autokosten,adac.autokreditdankeseite,adac.autokreditlandingpage,adac.autotest,adac.autovermietungproduktseite,adac.autoveskontaktfaq,adac.autovesleistungen,adac.autovesueberblick,adac.benzindiesel,adac.crashtest,adac.ecotest,adac.finanzdiensteproduktseite,adac.gebrauchtwagen,adac.homepage,adac.importexport,adac.infotestrateinsatz,adac.infotestratmotorrad,adac.infotestratoldtimer,adac.infotestratrecht,adac.infotestratstart,adac.infotestrattests,adac.infotestratunfall,adac.infotestratverkehr,adac.kamp1406,adac.kamp1411,adac.kfzsteuer,adac.kfzvers,adac.kraftstoffpreise,adac.kreditkarteantrag,adac.k

## 7.9) retransformer la prédiction qui est en chiffre en une prédiction en catégorie (homme/femme)

In [73]:
from pyspark.ml.feature import IndexToString

In [76]:
label_converter = IndexToString()

In [80]:
label_converter = label_converter.setInputCol("prediction")
label_converter = label_converter.setOutputCol("predicted_label")
label_converter = label_converter.setLabels(string_indexer_modele.labels)

## 7.9) création d'un pipeline

In [47]:
from pyspark.ml import Pipeline

In [48]:
pipeline = Pipeline()

In [83]:
stages = [string_indexer_modele  , # one hot encoding
          vector_assembler       , # une colonne avec toutes les colonnes de features
          modele_decision_tree   , # prediction via le modèle déjà entraîné
          label_converter        ] # conversion de la prédiction en label

In [85]:
pipeline = pipeline.setStages(stages)

In [86]:
pipeline_modele = pipeline.fit(jeu_d_apprentissage)

In [88]:
predictions = pipeline_modele.transform(jeu_de_test.drop("features"))

In [91]:
predictions.select("prediction", "predicted_label").show(5)

+----------+---------------+
|prediction|predicted_label|
+----------+---------------+
|       0.0|          femme|
|       0.0|          femme|
|       1.0|          homme|
|       0.0|          femme|
|       0.0|          femme|
+----------+---------------+
only showing top 5 rows



### Le pipeline effectue à la suite les tâches de (1) one-hot encoding, (2) d'assemblage des features dans un vecteur, (3) de prédiction et de (4) conversion des prédictions en label via une seule fonction "transform" : 4 étapes en un seul appel de fonction !  

### 7.9.1) sauvegarde du pipeline pour envoi dans un système de production

In [92]:
pipeline_modele.save("pipeline_prediction") 

# ------------------------ ANNEXES -------------------------

# Dans jupyter il est possible de faire des appels systèmes avec le point d'exclamation

# Find your ip => ip a 

In [45]:
!ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: eth0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 00:0d:3a:22:ec:95 brd ff:ff:ff:ff:ff:ff
    inet 10.0.0.4/20 brd 10.0.15.255 scope global eth0
       valid_lft forever preferred_lft forever
    inet6 fe80::20d:3aff:fe22:ec95/64 scope link 
       valid_lft forever preferred_lft forever


# Télécharger un fichier

In [14]:
!curl -O https://raw.githubusercontent.com/romainjouin/2021_esilv_spark/master/cycliste_debug.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48.1M  100 48.1M    0     0  21.2M      0  0:00:02  0:00:02 --:--:-- 21.2M


In [17]:
!curl -O https://raw.githubusercontent.com/romainjouin/2021_esilv_spark/master/base_x.csv.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.1M  100 13.1M    0     0  17.9M      0 --:--:-- --:--:-- --:--:-- 17.9M


In [18]:
!ls -tl

total 62788
-rw-rw-r-- 1 student student 13791612 Dec 18 09:01 base_x.csv.zip
-rw-rw-r-- 1 student student     8822 Dec 18 08:59 Untitled.ipynb
-rw-rw-r-- 1 student student 50484560 Dec 18 08:57 cycliste_debug.csv


# installation de unzip => sudo apt install unzip

In [21]:
!curl -O https://raw.githubusercontent.com/romainjouin/2021_esilv_spark/master/base_y.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1657k  100 1657k    0     0  5928k      0 --:--:-- --:--:-- --:--:-- 5940k


In [22]:
!ls

Untitled.ipynb	base_x.csv	base_y.csv
__MACOSX	base_x.csv.zip	cycliste_debug.csv
